In [ ]:
import os
import json
import numpy as np
import matplotlib as plt
data_dir = 'signData'

train_dir = os.path.join(data_dir,'train')
val_dir = os.path.join(data_dir,'valid')

output_dir = os.path.join(data_dir,'nptxt_lite')
weight_dir = os.path.join(data_dir,'weights')
val_output_dir = os.path.join(data_dir,"nptxt_lite_val")

train_landmark_dir = os.path.join(train_dir,'label','landmark')
train_morpheme_dir = os.path.join(train_dir,'label','morpheme')

val_landmark_dir = os.path.join(val_dir,'landmark')
val_morpheme_dir = os.path.join(val_dir,'morpheme')

In [ ]:
def weight_calc(F, a, b, x):
    fps=30
    a_frame = a * fps  # a를 프레임 단위로 변환
    b_frame = b * fps  # b를 프레임 단위로 변환
    x_frame = x * fps  # x(영상의 길이)를 프레임 단위로 변환
    if F < a_frame:
        return 0
    elif a_frame <= F < a_frame + 2 * fps:
        return (F - a_frame) / (2 * fps)
    elif a_frame + 2 * fps <= F < b_frame:
        return 1
    elif b_frame <= F <= x_frame:
        return 1 - ((F - b_frame) / (x_frame - b_frame)) * 0.5
    else:
        return 0.5
	

In [ ]:
#train weight data
# start_person="01"
# start_word="0001"
# #사람별 데이터(01~16)
# for person in os.listdir(train_landmark_dir):
#     #중간부터 다시 데이터 변환시 체크포인트
#     if int(person) < int(start_person):
#         continue
        
#     #변환 데이터 저장시 사람 폴더 생성(01~16)
#     person_output_path = os.path.join(weight_dir, str(int(person)))
#     os.makedirs(person_output_path, exist_ok=True)
    
#     #각 단어별 좌표값 및 단어 뜻 순회. 
#     for word_coords, word_morpheme in zip(os.listdir(os.path.join(train_landmark_dir, person)), os.listdir(os.path.join(train_morpheme_dir, person))):
#         #중간부터 다시 데이터 변환시 체크 포인트
#         if int(word_coords[11:15]) < int(start_word):
#             continue
#         #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
#         #단어 별 뜻, 단어의 할당 넘버, 영상 길이와 영상 내 수어 구간 추출
#         if "F" in word_morpheme:
#             file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
#             morpheme_file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
#             with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
#                 data = json.load(morpheme_file)
#                 try:
#                     num = data['metaData']['name'][11:15]
#                     start_sign = data["data"][0]["start"]
#                     end_sign = data["data"][0]["end"]
#                     duration = data["metaData"]["duration"]
#                 #결측치 오류 검출   
#                 except IndexError as e:
#                     name = False
#                     print(f"Error reading {morpheme_file_path}: {e}")
#                     continue
        
#         #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
#         if "F" in word_coords and name is not False:
#             wordWeight = []
#             #단어 영상의 각 프레임별 좌표값 추출
#             for frame in os.listdir(os.path.join(train_landmark_dir, person, word_coords)):
#                 frame_num = int(frame[25:37])
#                 # 가중치 계산만 수행
#                 wordWeight.append(weight_calc(frame_num, start_sign, end_sign, duration))
            
#             # 결과를 저장
#             time = [start_sign, end_sign, duration]
#             word_output_path = os.path.join(person_output_path, f'{num}.npz')
#             np.savez(word_output_path, weight=wordWeight, time=time)
#             print(f"Saved weights for {word_output_path}")
#         else:
#             continue

In [ ]:
#train data
def pre_data(landmark_dir,morpheme_dir,output_dir):
        import numpy as np
        start_person="01"
        start_word=0
        #사람별 데이터(01~16)
        for person in sorted(os.listdir(landmark_dir)):
            #중간부터 다시 데이터 변환시 체크포인트
            if int(person) < int(start_person):
                continue
                
            #변환 데이터 저장시 사람 폴더 생성(01~16)
            person_output_path = os.path.join(output_dir, str(int(person)))
            os.makedirs(person_output_path, exist_ok=True)
            
            #각 단어별 좌표값 및 단어 뜻 순회. 
            for word_coords,word_morpheme in zip(sorted(os.listdir(os.path.join(landmark_dir, person))),sorted(os.listdir(os.path.join(morpheme_dir, person)))):
                #중간부터 다시 데이터 변환시 체크 포인트
                if int(word_coords[11:15]) <start_word:
                    continue
                elif int(word_coords[11:15]) == start_word:
                    start_word=0
                #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
                #단어 별 뜻, 단어의 할당 넘버,영상 길이와 영상 내 수어 구간 추출
                file_path = os.path.join(morpheme_dir, person, word_morpheme)
                morpheme_file_path = os.path.join(morpheme_dir, person, word_morpheme)
                with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                    data = json.load(morpheme_file)
                    try:
                        name = data['data'][0]['attributes'][0]['name']
                        num = data['metaData']['name'][11:15]
                        start_sign = data["data"][0]["start"]
                        end_sign = data["data"][0]["end"]
                        duration = data["metaData"]["duration"]
                    #결측치 오류 검출   
                    except IndexError as e:
                        name=False
                        print(f"Error reading {morpheme_file_path}: {e}")
                        continue
                #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
                if name is not False:
                    wordCoordL = np.empty((0, 4, 5, 3))  
                    wordCoordR = np.empty((0, 4, 5, 3))
                    wordCoordP = np.empty((0, 3, 10))
                    wordWeight = []
                    #단어 영상의 각 프레임별 좌표값 추출            
                    for frame in sorted(os.listdir(os.path.join(landmark_dir, person, word_coords))):
                        
                        file_path = os.path.join(landmark_dir, person, word_coords, frame)
                        frame_num=int(frame[25:37])
                        with open(file_path, 'r') as json_file:
                            data = json.load(json_file)
                            lh_points = data['people']['hand_left_keypoints_3d']
                            rh_points = data['people']['hand_right_keypoints_3d']
                            p_points = data['people']['pose_keypoints_3d']
                            #포즈 좌표값 추출
                            #1920x1080영상을 메디어파이프의 특성상 영상의 중심을 기준으로 1080x1080으로 잘라서 기존 죄표를 픽셀 값으로 변환 후 
                            # 다시 미디어 파이프와 같은 스케일인 0~1사이 값으로 정규화
                            #z좌표는 미디어파이프의 기준이 되는 랜드마크의 좌표 활용하여 계산
                            
                            preFrameCoordP = np.array([[(960 * p_points[i] + 960), #중앙 랜드마크 x 0.5되도록  
                                                        (1080 * p_points[i + 1] + 540),
                                                        (p_points[32 + 2] - p_points[i + 2]) / 10]
                                                    for i in range(0, len(p_points), 4)], dtype=np.float32)

                            preFrameCoordL = np.array([[(960 * lh_points[i] + 960),
                                                        (1080 * lh_points[i + 1] + 540),
                                                        (lh_points[2] - lh_points[i + 2]) / 10]
                                                    for i in range(4, len(lh_points), 4)], dtype=np.float32)

                            preFrameCoordR = np.array([[(960 * rh_points[i] + 960),
                                                        (1080 * rh_points[i + 1] + 540),
                                                        (rh_points[2] - rh_points[i + 2]) / 10]
                                                    for i in range(4, len(rh_points), 4)], dtype=np.float32)
                            
                            
                            #포즈 좌표에서 하반신 랜드마크 제외
                            preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]
                            
                            #랜드마크 값들을 모델 트레이닝에 적합한 모양으로 재배열.
                            frameCoordL=preFrameCoordL.reshape(5,4,3).transpose(1,0,2)[::-1]
                            frameCoordR=preFrameCoordR.reshape(5,4,3).transpose(1,0,2)[::-1]
                            frameCoordP=preFrameCoordP.T

                            import numpy as np


                            # 1번 인덱스의 x, y 좌표
                            x_center = frameCoordP[0][1]
                            y_center = frameCoordP[1][1]

                            # 1080x1080 크롭 영역을 정하기 위해 크롭 영역의 좌상단 좌표 계산
                            x_min = max(0, x_center - 540)
                            y_min = max(0, y_center - 540)

                            # 크롭 영역이 1920x1080을 초과하지 않도록 조정
                            x_min = min(x_min, 1920 - 1080)
                            y_min = min(y_min, 1080 - 1080)

                            # 크롭 영역의 우상단 좌표 계산
                            x_max = x_min + 1080
                            y_max = y_min + 1080

                            # 좌표들을 크롭 영역에 맞게 이동
                            frameCoordP[0] -= x_min
                            frameCoordP[1] -= y_min
                            frameCoordP[0] /= 1080
                            frameCoordP[1] /= 1080

                            frameCoordP[0] = np.clip(frameCoordP[0], 0, 1)
                            frameCoordP[1] = np.clip(frameCoordP[1], 0, 1)

                            
                            def min_cal(coords):
                                coord_x_min = np.min(coords[:, :, 0]if coords.ndim == 3 else coords[:, 0])
                                coord_x_cal = np.max(coords[:, :, 0]if coords.ndim == 3 else coords[:, 0]) - coord_x_min 

                                coord_y_min = np.min(coords[:, :, 1]if coords.ndim == 3 else coords[:, 1])
                                coord_y_cal = np.max(coords[:, :, 1]if coords.ndim == 3 else coords[:, 1]) - coord_y_min 
                                
                                return [coord_x_min,coord_y_min,np.min(frameCoordL[:, :, 2]if coords.ndim == 3 else coords[:, 2])],[coord_x_cal,coord_y_cal]

                            coordL_min,coordL_cal=min_cal(frameCoordL)
                            coordR_min,coordR_cal=min_cal(frameCoordR)

                            frameCoordL = (frameCoordL - coordL_min) / (coordL_cal[0] if coordL_cal[0] > coordL_cal[1] else coordL_cal[1])
                            frameCoordR = (frameCoordR - coordR_min) / (coordR_cal[0] if coordR_cal[0] > coordR_cal[1] else coordR_cal[1])
                            
                            #한 단어의 프레임 별 좌표를 하나의 넘파이 배열로 저장
                            wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
                            wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
                            wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)
                            wordWeight.append(weight_calc(frame_num,start_sign,end_sign,duration))
                            
                    label=[name,num,start_sign,end_sign,duration]
                    word_output_path = os.path.join(person_output_path, f'{num}.npz')
                    np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label, weight=wordWeight)
                    print(f"Saved {word_output_path}")

                else:
                    continue


In [ ]:
pre_data(val_landmark_dir,val_morpheme_dir,val_output_dir)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
plt.scatter(frameCoordP[0], frameCoordP[1], color='blue', label="Transformed Points")
plt.scatter([frameCoordP[0][1]], [frameCoordP[1][1]], color='red', label="Center Point (Index 1)")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.axhline(0.5, color='gray', linestyle='--', linewidth=0.5)
plt.axvline(0.5, color='gray', linestyle='--', linewidth=0.5)
plt.title("Normalized Coordinates after Cropping and Centering")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# #val data
# start_person="01"
# start_word="0001"
# #사람별 데이터(01~16)
# for person in os.listdir(val_landmark_dir):  # train_landmark_dir -> val_landmark_dir
#     #중간부터 다시 데이터 변환시 체크포인트
#     if int(person) < int(start_person):
#         continue
        
#     #변환 데이터 저장시 사람 폴더 생성(01~16)
#     person_output_path = os.path.join(val_output_dir, str(int(person)))
#     os.makedirs(person_output_path, exist_ok=True)
    
#     #각 단어별 좌표값 및 단어 뜻 순회. 
#     for word_coords,word_morpheme in zip(os.listdir(os.path.join(val_landmark_dir, person)),os.listdir(os.path.join(val_morpheme_dir, person))):  # train_landmark_dir -> val_landmark_dir, train_morpheme_dir -> val_morpheme_dir
#         #중간부터 다시 데이터 변환시 체크 포인트
#         if int(word_coords[11:15]) < int(start_word):
#             continue
#         #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
#         #단어 별 뜻, 단어의 할당 넘버,영상 길이와 영상 내 수어 구간 추출
#         if "F" in word_morpheme:
#             file_path = os.path.join(val_morpheme_dir, person, word_morpheme)  # train_morpheme_dir -> val_morpheme_dir
#             morpheme_file_path = os.path.join(val_morpheme_dir, person, word_morpheme)  # train_morpheme_dir -> val_morpheme_dir
#             with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
#                 data = json.load(morpheme_file)
#                 try:
#                     name = data['data'][0]['attributes'][0]['name'].replace('\n', '')
#                     num = data['metaData']['name'][11:15]
#                 #결측치 오류 검출   
#                 except IndexError as e:
#                     name = False
#                     print(f"Error reading {morpheme_file_path}: {e}")
#                     continue
#         #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
#         if "F" in word_coords and name is not False:
#             wordCoordL = np.empty((0, 4, 5, 3))  
#             wordCoordR = np.empty((0, 4, 5, 3))
#             wordCoordP = np.empty((0, 3, 10))
#             wordWeight = []
#             #단어 영상의 각 프레임별 좌표값 추출            
#             for frame in os.listdir(os.path.join(val_landmark_dir, person, word_coords)):  # train_landmark_dir -> val_landmark_dir
#                 file_path = os.path.join(val_landmark_dir, person, word_coords, frame)  # train_landmark_dir -> val_landmark_dir
#                 frame_num = int(frame[25:37])
#                 with open(file_path, 'r') as json_file:
#                     data = json.load(json_file)
#                     lh_points = data['people']['hand_left_keypoints_3d']
#                     rh_points = data['people']['hand_right_keypoints_3d']
#                     p_points = data['people']['pose_keypoints_3d']
#                     #포즈 좌표값 추출 및 정규화 코드 동일
#                     preFrameCoordP = np.array([[(960 * p_points[i] + 960 - 420) / (1500 - 420), 
#                                                 (1080 * p_points[i + 1] + 540) / 1080,
#                                                 (p_points[32 + 2] - p_points[i + 2]) / 10]
#                                                for i in range(0, len(p_points), 4)], dtype=np.float32)

#                     preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
#                                                 (1080 * lh_points[i + 1] + 540) / 1080,
#                                                 (lh_points[2] - lh_points[i + 2]) / 10]
#                                                for i in range(4, len(lh_points), 4)], dtype=np.float32)

#                     preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
#                                                 (1080 * rh_points[i + 1] + 540) / 1080,
#                                                 (rh_points[2] - rh_points[i + 2]) / 10]
#                                                for i in range(4, len(rh_points), 4)], dtype=np.float32)
                    
#                     #포즈 좌표에서 하반신 랜드마크 제외
#                     preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]
                    
#                     #랜드마크 값들을 모델 트레이닝에 적합한 모양으로 재배열.
#                     frameCoordL = preFrameCoordL.reshape(5, 4, 3).transpose(1, 0, 2)[::-1]
#                     frameCoordR = preFrameCoordR.reshape(5, 4, 3).transpose(1, 0, 2)[::-1]
#                     frameCoordP = preFrameCoordP.T
#                     #한 단어의 프레임 별 좌표를 하나의 넘파이 배열로 저장
#                     wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
#                     wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
#                     wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)

                    
#             label = [int(person),num,name]
#             word_output_path = os.path.join(person_output_path, f'{num}.npz')
#             np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label)
#             print(f"Saved {word_output_path}")
#         else:
#             continue

In [ ]:


import os
import json
import numpy as np

data_dir = 'signData'

train_dir = os.path.join(data_dir,'train')
val_dir = os.path.join(data_dir,'valid')

output_dir = os.path.join(data_dir,'nptxt_CL')
val_output_dir = os.path.join(data_dir,"nptxt_CL_val")
# weight_dir = os.path.join(data_dir,'weights')

train_landmark_dir = os.path.join(train_dir,'label','landmark')
train_morpheme_dir = os.path.join(train_dir,'label','morpheme')

val_landmark_dir = os.path.join(val_dir,'landmark')
val_morpheme_dir = os.path.join(val_dir,'morpheme')


def getoutputdir(type="train"):
    return val_output_dir if type=='val' else output_dir

with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    words_dicts = json.load(json_file)

def load_data(file_name,type="train"):
    path=f"{getoutputdir(type)}/{file_name}"

    data = np.load(path)

    #좌표값 로드
    wordCoordL = data['wordCoordL']
    wordCoordR = data['wordCoordR']
    wordCoordP = data['wordCoordP']
    wordWeight = data['weight']
    #단어 뜻 호출
    ans = data['label'][0]
    ans = ans.replace('\n', '')
    #해당 단어의 value 호출
    if ans[-1].isdigit():
        ans = ans[:-1]
    # label = words_dicts[ans]
    label=ans
    if type=='train':
        return wordCoordL, wordCoordR, wordCoordP, label ,wordWeight
    else:
        return wordCoordL, wordCoordR, wordCoordP, label




def load_word(person, start, num):
    coordLs = []
    coordRs = []
    coordPs = []
    labels = []
    checks = []
    for WNum in range(start, start + num):
        wordCoordL, wordCoordR, wordCoordP, label = load_data(person, WNum)
        coordLs.append(wordCoordL)
        coordRs.append(wordCoordR)
        coordPs.append(wordCoordP)
        labels.append(int(label[0]))
        checks.append((label[1:3]))

    return coordLs, coordRs, coordPs, labels, checks




In [ ]:


# import os
# import json
# import numpy as np

# data_dir = 'signData'

# train_dir = os.path.join(data_dir,'train')
# val_dir = os.path.join(data_dir,'valid')

# output_dir = os.path.join(data_dir,'nptxt')
# val_output_dir = os.path.join(data_dir,"nptxt_val")
# weight_dir = os.path.join(data_dir,'weights')

# train_landmark_dir = os.path.join(train_dir,'label','landmark')
# train_morpheme_dir = os.path.join(train_dir,'label','morpheme')

# val_landmark_dir = os.path.join(val_dir,'landmark')
# val_morpheme_dir = os.path.join(val_dir,'morpheme')


# def getoutputdir(type="train"):
#     return val_output_dir if type=='val' else output_dir

# with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
#     words_dicts = json.load(json_file)


# def load_data(file_name,type="train"):
#     path=f"{getoutputdir(type)}/{file_name}"
#     w_path=f"{weight_dir}/{file_name}"
#     data = np.load(path)
#     weight_data = np.load(w_path)
#     #좌표값 로드
#     wordCoordL = data['wordCoordL']
#     wordCoordR = data['wordCoordR']
#     wordCoordP = data['wordCoordP']
#     # weight = data['weight']
#     weight = weight_data['weight']
#     #단어 뜻 호출
#     ans = data['label'][2]
#     ans = ans.replace('\n', '')
#     #해당 단어의 value 호출
#     if ans[-1].isdigit():
#         ans = ans[:-1]
#     label = words_dicts[ans]
#     if type=='train':
#         return wordCoordL, wordCoordR, wordCoordP, label ,weight
#     else:
#         return wordCoordL, wordCoordR, wordCoordP, label




# def load_word(person, start, num):
#     coordLs = []
#     coordRs = []
#     coordPs = []
#     labels = []
#     checks = []
#     for WNum in range(start, start + num):
#         wordCoordL, wordCoordR, wordCoordP, label = load_data(person, WNum)
#         coordLs.append(wordCoordL)
#         coordRs.append(wordCoordR)
#         coordPs.append(wordCoordP)
#         labels.append(int(label[0]))
#         checks.append((label[1:3]))

#     return coordLs, coordRs, coordPs, labels, checks




In [ ]:
wordCoordL, wordCoordR, wordCoordP, label ,wordWeight=load_data("1/0001.npz")
print(label)

In [ ]:
wordWeight.shape

In [ ]:
def number_to_korean(n):
    units = ['', '만', '억', '조', '경']
    nums = ['영', '일', '이', '삼', '사', '오', '육', '칠', '팔', '구']
    small_units = ['', '십', '백', '천']

    def split_by_thousands(num):
        chunks = []
        while num > 0:
            chunks.append(num % 10000)
            num //= 10000
        return chunks

    def convert_chunk(chunk):
        result = []
        for i, digit in enumerate(str(chunk).zfill(4)):
            if digit != '0':
                result.append(nums[int(digit)] + small_units[3 - i])
        return ''.join(result)

    if n == 0:
        return nums[0]

    chunks = split_by_thousands(n)
    korean_number = []

    for i, chunk in enumerate(chunks):
        if chunk > 0:
            korean_number.append(convert_chunk(chunk) + units[i])

    result = ''.join(reversed(korean_number))
    result = result.replace("일천", "천").replace("일만", "만").replace("일십","십").replace("일백","백")
    return result
# numbers_01=[number_to_korean(n) for n in range(11,210)]
# numbers_02=[number_to_korean(n) for n in range(210,400,10)]
# numbers_04=[number_to_korean(n) for n in range(400,1000,50)]
# numbers_05=[number_to_korean(n) for n in range(1000,10000,500)]
# numbers_06=[number_to_korean(n) for n in range(10000,100000,5000)]
# numbers_07=[number_to_korean(n) for n in range(100000,1000000,100000)]
# numbers_08=[number_to_korean(n) for n in range(1000000,10000000,1000000)]
# numbers_09=[number_to_korean(n) for n in range(10000000,100000000,10000000)]
# numbers_10=[number_to_korean(n) for n in range(100000000,1000000000,50000000)]
# numbers_11=[number_to_korean(n) for n in range(1000000000,10000000000,500000000)]
# numbers_12=[number_to_korean(n) for n in range(10000000000,100000000000,1000000000)]
ranges = [
    range(11, 210),
    range(210, 400, 10),
    range(400, 1000, 50),
    range(1000, 10000, 500),
    range(10000, 100000, 5000),
    range(100000, 1000000, 50000),
    range(1000000, 10000000, 500000),
    range(10000000, 100000000, 5000000),
    range(100000000, 1000000000, 50000000),
    range(1000000000, 10000000000, 500000000),
    range(10000000000, 100000000000, 10000000000),
]

numbers = [number_to_korean(n) for r in ranges for n in r]
numbers.append("천억")
numbers.append("일조")






In [ ]:
black_names=["지부","센터","협회","지회","시청","특별법","경기대회",'대학교','도청','청장','0만원']

def black_list(name):
    if name == "시청":
        True
    elif name in numbers:
        return True
    else:
        for b in black_names:
            if b in name:
                return True
        return False

In [ ]:
#make lable



person_blacklist=[]
word_dict={}
num=1
for person in os.listdir(train_morpheme_dir):
    for word_morpheme in os.listdir(os.path.join(train_morpheme_dir, person)):
        if "F" in word_morpheme:
            file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            morpheme_file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    name = data['data'][0]['attributes'][0]['name']
                    name = name.replace("\n", "")
                    if name[-1].isdigit():
                        name = name[:-1]
                    if name in word_dict or black_list(name) or name=='.절':
                        continue
                    else:
                        print(name)
                        word_dict[name]=num
                        num+=1
                except IndexError as e:
                    name=False
                    continue
               
    with open('wordtonum_lite.json', 'w') as json_file:
        json.dump(word_dict, json_file)

In [3]:
with open('wordtonum_lite.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(len(data))
    

1879


In [5]:
with open('wordtonum_lite.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(data['.절'])
    

KeyError: '.절'

In [ ]:
import os
import shutil
import json

destination_folder = os.path.join(data_dir, "nptxt_lite_val")

# JSON 파일 로드
with open('wordtonum_lite.json', 'r', encoding="UTF8") as json_file:
    word_list = json.load(json_file)

# 대상 폴더가 없으면 생성
os.makedirs(destination_folder, exist_ok=True)

# 폴더 내 파일 확인 및 복사
for person in os.listdir(val_output_dir):
    for filename in sorted(os.listdir(os.path.join(val_output_dir, person))):
        if filename[0]==".":
            continue
        source_file = os.path.join(person, filename)
        print(source_file)
        
        # 파일 정보 읽기
        _, _, _, word = load_data(source_file,"val")
        print(word)
        
        # 단어가 리스트에 있으면 복사
        if word in word_list:
            copy_path = os.path.join(destination_folder, person)
            os.makedirs(copy_path, exist_ok=True)
            shutil.copy2(os.path.join(val_output_dir, source_file), copy_path)  # 파일 복사
            print(f"Copied: {filename}")